# 基于RNN 的字符生成

## RNN

![RNN](../Docs/RNN-Core.png)

* 更新隐藏状态
$$\boldsymbol{H}t = \phi(\boldsymbol{X}t \boldsymbol{W}{xh} + \boldsymbol{H}{t-1} \boldsymbol{W}_{hh} + \boldsymbol{b}_h).$$

* 计算输出
$$\boldsymbol{O}_t = \boldsymbol{H}t \boldsymbol{W}{hq} + \boldsymbol{b}_q.$$

## 困惑度
通常使用困惑度（complexity）来评价语言模型的好坏。

**困惑度是对交叉损失函数做指数运算后得到的值。**

**任何一个有效模型的困惑度必须小于类别个数。**

In [1]:
import time
import math
import numpy as np
import torch
from torch import nn, optim
import torch.nn.functional as F

import numpy as np
import pickle
import zipfile
import copy
import random

import utils as d2l

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
class TextTokenizer(object):
    def __init__(self, text=None, max_vocab=None, filename=None):
        if filename is not None:
            with open(filename, 'rb') as f:
                self.vocab = pickle.load(f)
        else:
            corpus_chars = set(text)
            # max_vocab_process
            vocab_count = {}
            for word in corpus_chars:
                if vocab_count.get(word) is None:
                    vocab_count[word] = 1
                else:
                    vocab_count[word] += 1

            vocab_count_list = []
            for word in vocab_count:
                vocab_count_list.append((word, vocab_count[word]))
            # sort count with number
            vocab_count_list.sort(key=lambda x: x[1], reverse=True)
            if max_vocab is not None and len(vocab_count_list) > max_vocab:
                vocab_count_list = vocab_count_list[:max_vocab]
            vocab = [x[0] for x in vocab_count_list]
            self.vocab = vocab

        self.char_to_index = {c: i for i, c in enumerate(self.vocab)}
        self.index_to_char = dict(enumerate(self.vocab))

    @property
    def vocab_size(self):
        return len(self.vocab) + 1

    def word_to_index(self, word):
        if word in self.char_to_index:
            return self.char_to_index[word]
        else:
            return len(self.vocab)

    def index_to_word(self, index):
        if index == len(self.vocab):
            return '<unk>'
        elif index < len(self.vocab):
            return self.char_to_index[index]
        else:
            raise Exception('Unknown index!')

    def text_to_array(self, text):
        arr = []
        for word in text:
            arr.append(self.word_to_index(word))
        return np.array(arr)

    def array_to_text(self, arr):
        words = []
        for index in arr:
            words.append(self.index_to_word(index))
        return "".join(words)

    def save_to_file(self, filename):
        with open(filename, 'wb') as f:
            pickle.dump(self.vocab, f)

In [3]:
def data_loader_random(corpus_indices, seq_length, time_steps, device=None):
    num_samples = (len(corpus_indices) - 1) // time_steps
    epoch_size = num_samples // seq_length

    # shuffle sample
    samples_indices = list(range(num_samples))
    random.shuffle(samples_indices)

    # generator data
    for i in range(epoch_size):
        i = i * seq_length

        batch_incices = samples_indices[i: i + seq_length]
        x = [corpus_indices[indices: indices + time_steps] for indices in batch_incices]
        y = [corpus_indices[indices + 1: indices + time_steps + 1] for indices in batch_incices]

        x = torch.tensor(x, dtype=torch.float32).view(seq_length, time_steps)
        y = torch.tensor(y, dtype=torch.float32).view(seq_length, time_steps)

        yield x, y

In [4]:
def data_loader_consecutive(corpus_indices, seq_length, time_steps):

    data_len = len(corpus_indices)

    seq_size = data_len // seq_length

    # resize to => (seq_length, seq_size)
    corpus_indices = np.array(corpus_indices[: seq_size * seq_length], dtype=np.float).reshape((seq_length, -1))

    epoch_size = (seq_size - 1) // time_steps

    # generator data
    np.random.shuffle(corpus_indices)

    # convert to torch tensor
    torch_indices = torch.tensor(corpus_indices, dtype=torch.float32).view(seq_length, seq_size)
    for i in range(epoch_size):
        i = i * time_steps
        x = torch_indices[:, i: i + time_steps]
        y = torch_indices[:, i + 1: i + time_steps + 1]

        yield x, y

In [5]:
with zipfile.ZipFile('../Datasets/jaychou_lyrics/jaychou_lyrics.txt.zip') as zin:
    with zin.open('jaychou_lyrics.txt') as f:
        corpus_chars = f.read().decode('utf-8')  # corpus

corpus_chars = corpus_chars.replace('\n', ' ').replace('\t', ' ')
corpus_chars = corpus_chars[: 20000]

tokenizer = TextTokenizer(text=corpus_chars, max_vocab=None)

vocab = tokenizer.vocab
char_to_index = tokenizer.char_to_index
index_to_char = tokenizer.index_to_char
vocab_size = tokenizer.vocab_size

corpus_indices = tokenizer.text_to_array(corpus_chars)

In [6]:
def onehot(X, n_class):
    # X shape: (batch, seq_len), output: seq_len elements of (batch, n_class)

    return [F.one_hot(X[:, i].to(torch.int64), n_class).to(dtype=torch.float32, device=device) for i in range(X.shape[1])]

X = torch.arange(10).view(2, 5)
inputs = onehot(X, vocab_size)
print(len(inputs), inputs[0].shape)

5 torch.Size([2, 1448])


In [7]:
num_inputs, num_hiddens, num_outputs = vocab_size, 256, vocab_size
print('will use', device)

def get_params():
    def _one(shape):
        ts = torch.tensor(np.random.normal(0, 0.01, size=shape), device=device, dtype=torch.float32)
        return torch.nn.Parameter(ts, requires_grad=True)

    # 隐藏层参数
    W_xh = _one((num_inputs, num_hiddens))
    W_hh = _one((num_hiddens, num_hiddens))
    b_h = torch.nn.Parameter(torch.zeros(num_hiddens, device=device), requires_grad=True)
    # 输出层参数
    W_hq = _one((num_hiddens, num_outputs))
    b_q = torch.nn.Parameter(torch.zeros(num_outputs, device=device), requires_grad=True)
    return nn.ParameterList([W_xh, W_hh, b_h, W_hq, b_q])

will use cuda


In [8]:
def init_rnn_state(seq_length, num_hiddens, device):
    return (torch.zeros((seq_length, num_hiddens), device=device),)

In [9]:
def rnn(inputs, state, params):
    # inputs和outputs皆为num_steps个形状为(seq_length, vocab_size)的矩阵
    W_xh, W_hh, b_h, W_hq, b_q = params
    H, = state
    outputs = []
    for X in inputs:
        H = torch.tanh(torch.matmul(X, W_xh) + torch.matmul(H, W_hh) + b_h)
        Y = torch.matmul(H, W_hq) + b_q
        outputs.append(Y)
    return outputs, (H,)

In [10]:
X = torch.arange(10).view(2, 5)
state = init_rnn_state(X.shape[0], num_hiddens, device)
inputs = onehot(X.to(device), vocab_size)
print(inputs[0].size())
params = get_params()
outputs, state_new = rnn(inputs, state, params)
print(len(outputs), outputs[0].shape, state_new[0].shape)

torch.Size([2, 1448])
5 torch.Size([2, 1448]) torch.Size([2, 256])


In [11]:
def grad_clipping(params, theta, device):
    norm = torch.tensor([0.0], device=device)
    for param in params:
        norm += (param.grad.data ** 2).sum()
    norm = norm.sqrt().item()
    if norm > theta:
        for param in params:
            param.grad.data *= (theta / norm)

In [12]:
def predict_rnn(prefix, num_chars, rnn, params, init_rnn_state,
                num_hiddens, vocab_size, device, idx_to_char, char_to_idx):
    state = init_rnn_state(1, num_hiddens, device)
    output = [char_to_idx[prefix[0]]]
    for t in range(num_chars + len(prefix) - 1):
        # 将上一时间步的输出作为当前时间步的输入
        X = onehot(torch.tensor([[output[-1]]], device=device), vocab_size)
        # 计算输出和更新隐藏状态
        (Y, state) = rnn(X, state, params)
        # 下一个时间步的输入是prefix里的字符或者当前的最佳预测字符
        if t < len(prefix) - 1:
            output.append(char_to_idx[prefix[t + 1]])
        else:
            output.append(int(Y[0].argmax(dim=1).item()))
    return ''.join([idx_to_char[i] for i in output])

In [13]:
p = predict_rnn('分开', 10, rnn, params, init_rnn_state, num_hiddens, vocab_size,
                    device, index_to_char, char_to_index)

print(p)

分开瓜拿存稳任坛探右力丁


In [14]:
def train_and_predict_rnn(rnn, get_params, init_rnn_state, num_hiddens,
                          vocab_size, device, corpus_indices, idx_to_char,
                          char_to_idx, is_random_iter, num_epochs, num_steps,
                          lr, clipping_theta, seq_length, pred_period,
                          pred_len, prefixes):
    if is_random_iter:
        data_iter_fn = d2l.data_iter_random
    else:
        data_iter_fn = d2l.data_iter_consecutive
    params = get_params()
    loss = nn.CrossEntropyLoss()

    for epoch in range(num_epochs):
        if not is_random_iter:  # 如使用相邻采样，在epoch开始时初始化隐藏状态
            state = init_rnn_state(seq_length, num_hiddens, device)
        l_sum, n, start = 0.0, 0, time.time()
        data_iter = data_iter_fn(corpus_indices, seq_length, num_steps, device)
        for X, Y in data_iter:
            if is_random_iter:  # 如使用随机采样，在每个小批量更新前初始化隐藏状态
                state = init_rnn_state(seq_length, num_hiddens, device)
            else:
                # 否则需要使用detach函数从计算图分离隐藏状态, 这是为了
                # 使模型参数的梯度计算只依赖一次迭代读取的小批量序列(防止梯度计算开销太大)
                for s in state:
                    s.detach_()

            inputs = onehot(X, vocab_size)
            # outputs有num_steps个形状为(seq_length, vocab_size)的矩阵
            (outputs, state) = rnn(inputs, state, params)
            # 拼接之后形状为(num_steps * seq_length, vocab_size)
            outputs = torch.cat(outputs, dim=0)
            # Y的形状是(seq_length, num_steps)，转置后再变成长度为
            # batch * num_steps 的向量，这样跟输出的行一一对应
            y = torch.transpose(Y, 0, 1).contiguous().view(-1)
            # 使用交叉熵损失计算平均分类误差
            l = loss(outputs, y.long())

            # 梯度清0
            if params[0].grad is not None:
                for param in params:
                    param.grad.data.zero_()
            l.backward()
            grad_clipping(params, clipping_theta, device)  # 裁剪梯度
            d2l.sgd(params, lr, 1)  # 因为误差已经取过均值，梯度不用再做平均
            l_sum += l.item() * y.shape[0]
            n += y.shape[0]

        if (epoch + 1) % pred_period == 0:
            print('epoch %d, perplexity %f, time %.2f sec' % (
                epoch + 1, math.exp(l_sum / n), time.time() - start))
            for prefix in prefixes:
                print(' -', predict_rnn(prefix, pred_len, rnn, params, init_rnn_state,
                                        num_hiddens, vocab_size, device, idx_to_char, char_to_idx))

In [15]:
num_epochs, num_steps, seq_length, lr, clipping_theta = 250, 35, 32, 1e2, 1e-2
pred_period, pred_len, prefixes = 50, 50, ['分开', '不分开']

train_and_predict_rnn(rnn, get_params, init_rnn_state, num_hiddens,
                      vocab_size, device, corpus_indices, index_to_char,
                      char_to_index, True, num_epochs, num_steps, lr,
                      clipping_theta, seq_length, pred_period, pred_len,
                      prefixes)

epoch 50, perplexity 49.369783, time 0.22 sec
 - 分开 一直两步 我们了 一步两颗三步四颗 等天我们著的可 一直走剧 全小了空 在有了空 在在用空 在人用
 - 不分开 我只想起 你只会感到更加沮 别想我 别怪我 娘着我 单的我 别过 化什么 有什么  什么这样 我们
epoch 100, perplexity 7.568152, time 0.26 sec
 - 分开 一直好气 快使用双截棍 哼哼哈兮 快使用双截棍 哼哼哈兮 快使用双截棍 哼哼哈兮 快使用双截棍 哼
 - 不分开不能信天我 等不休我 爱你不能球更 爱天 他的古口 我想要回 你去的美 你知道 别不去 别影我的地球
epoch 150, perplexity 3.247775, time 0.25 sec
 - 分开 你以想要多了离开还就为 远和的世 已吹的日尘 用制的神乐多 我会想到你小 我爱不起 我知之好生活 
 - 不分开不要一天 从故的眼我面狂的可爱女人 温柔的让我疯狂的可爱女人 坏坏的让我疯狂的可爱女人 坏坏的让我疯
epoch 200, perplexity 2.319744, time 0.24 sec
 - 分开 单彻安 半什么 娘时了纳手代刚 干什么 干什么 想沉丹任手球真 干什么 干什么 想沉开任手球将 干
 - 不分开不 我想要再想 我不 我不 我不要再想你 不知不觉 你已经离开我 不知不觉 我跟了这节奏 后知后觉 
epoch 250, perplexity 2.092166, time 0.25 sec
 - 分开 单水就能们 我对悔远控 黑云在降落 我被它拖着走 为我会想睡难 别隔壁对板 有天让 古着 只烟为久
 - 不分开吗 我叫在再想 他不能逃妥 一朵事他 温暖下人觉 白色蜡习 温暖的背尘 银制茶壶 在回村 废墟的风牺
